In [1]:
import csv
import numpy as np # if not installed use "pip install numpy"
import pandas as pd #  if not installed use "pip install pandas"

ModuleNotFoundError: No module named 'pandas'

## load_dataset_module.py

In [5]:
# list of features to retrive from the dataset
features_to_retrieve = ['acousticness', 'artists', 'danceability', 'energy', 'id', 'liveness',
                            'loudness', 'name', 'popularity', 'speechiness', 'tempo','valence']


In [6]:
class Load_data():
    def ___init___(self):
        pass

    def load_dataset(self):
        """
        This fuction loads the dataset.csv and stores it in a dictionary,
        the dataset column names and its values being used as key-value pair in dictionary
        return:
            a dictionary format of the dataset
        """
        csv_data = pd.read_csv("data.csv", encoding='utf-8')
        csv_data = csv_data.loc[:, features_to_retrieve]
        csv_data['name'] = csv_data['name'].astype('str')
        csv_data['artists'] = csv_data['artists'].astype('str')
        csv_data['artists'] = csv_data['artists'].str[1:-1]
        #csv_data['artists'] = csv_data['artists'].str.replace(",","")
        csv_data['artists'] = csv_data['artists'].str.replace("[","", regex=False)
        csv_data['artists'] = csv_data['artists'].str.replace("]","", regex=False)
        csv_data['artists'] = csv_data['artists'].str.replace("'","")
        return csv_data



    def artist_music_func(self):
        """
        This function returns artist music dataframe and a dictionary that
        contains artists name, music name, and corresponding features
        """
        
        features = ['artists', 'acousticness', 'danceability', 'energy', 'liveness',
                                'loudness', 'popularity', 'speechiness', 'tempo', 'valence']
        data = self.load_dataset()
        artist_music = data[features]
        artist_music['artists'] = artist_music['artists'].astype('category')
        artist_music['artists_cat'] = artist_music["artists"].cat.codes
        artist_music = artist_music.set_index("artists_cat")
        artist_music_dict = artist_music.to_dict()['artists']
        artist_music = artist_music.drop(['artists'], axis=1)
        artist_music = artist_music.groupby(artist_music.index).mean()
        return artist_music, artist_music_dict

    def music_features_func(self):
        """
        This fuction returns  music_features dataset and a  dictionary contains music id, and their respective features.
        """
        features = ['name', 'acousticness', 'danceability', 'energy', 'liveness',
                                'loudness', 'popularity', 'speechiness', 'tempo', 'valence']
        data = self.load_dataset()
        music_features = data[features]
        music_features_dict = music_features.to_dict()['name']
        music_features = music_features.drop(['name'], axis=1)
        return music_features, music_features_dict
    
    def music_for_artist_func(self):
        """
       This fuction returns  music_features dataset and a  dictionary contains music id, and their respective features.

        """
        features = ['name','artists' ,'acousticness', 'danceability', 'energy', 'liveness',
                                'loudness', 'popularity', 'speechiness', 'tempo', 'valence']
        data = self.load_dataset()
        music_for_artist = data[features]
        music_for_artist['artists'] = music_for_artist['artists'].astype('category')
        music_for_artist['artists_cat'] = music_for_artist["artists"].cat.codes
        music_dict = music_for_artist.to_dict()['name']
        music_for_artist = music_for_artist.drop(['name', 'artists'], axis=1)
        return music_for_artist, music_dict


## similarity_module.py

In [7]:
from sklearn.metrics.pairwise import pairwise_distances
from load_dataset_module import Load_data

In [8]:
import numpy as np #if not install use "pip install numpy"
import pandas as pd
from sklearn.metrics.pairwise import pairwise_distances
from load_dataset_module import Load_data
import warnings
warnings.simplefilter(action='ignore')

#Created the load data Class to collect data 
ld =  Load_data()
class Similarity_comp():
    def ___init___(self):
        pass

    def _compute_similar(self, types ,iden , similarity_func, n):
        """
        type : type of dataframe needed for method
        similarity_func : Similarity algorithm to use
        iden: unique identifier for the Music or Artist. 
        n: number of most similar values to return 

        This method take an id and the a similarity metric
        then carries out mathematical computation using the numerical values of the ids
        using the similarity metric pass and return the top n result, .
        """
        #we use the different type of methods to determine 
        #the data we would collect from the load_dataset Class
        if types == "music":
            data, data_dict = ld.music_features_func()
           
        elif types == "artist":
            data, data_dict = ld.artist_music_func()
            
        elif types == "music for artist":
            data, data_dict = ld.music_for_artist_func()
        
        if types == "music":
            first = data[data.index == iden]
            first = first.values

        elif types == "artist" :
            first = data[data.index == iden]
            first = first.groupby(by=first.index).mean()
           
            first = first.values
        elif types == "music for artist":
            data = data[data["artists_cat"] == iden]
            first = data.groupby(by="artists_cat").mean()
            data = data.drop(["artists_cat"], axis=1)
            first = first.values
        
        second = data.values
        
     
        arr = pairwise_distances(first, second, metric=similarity_func)
        arr = arr.reshape(-1, 1)
        preds = pd.DataFrame(arr,index=data.index, columns=["similarity"])
        if types == "artist":
            preds["artist"] = preds.index.map(data_dict)
            preds = preds.reset_index(drop=True)
            
        if types == "music for artist" or types == "music" :
            preds["music"] = preds.index.map(data_dict)
            preds = preds.reset_index(drop=True)
                    

        preds = preds.sort_values(by="similarity")
        
        print(f"Top {n} number of similar {types} to {data_dict[iden]} are\n", preds.head(n))


    def compute_similar_music(self, music_id,  similarity_func, n):
        """
        music_id : the song id
        similarity_func =the similarity function 
        n : number of similar values to return 

        this method takes the music name, the similarity_metrics and number of similar 
        music to return. This values are then feed to the compute similar method and return result gotten from 
        the compute_similar method. 
        """

        return self._compute_similar(types="music",iden=music_id, similarity_func=similarity_func, n=n)
    
    def compute_similar_artist(self, artist_id, similarity_func, n):
        """
        artist_id : the artist id
        similarity_func =the similarity function 
        n : number of similar values to return 

        this method takes the artist name, the similarity_metrics and number of similar 
        music to return. This values are then feed to the compute similar method and return result gotten from 
        the compute_similar method. 
        """
        return self._compute_similar(types="artist", iden=artist_id, similarity_func=similarity_func, n=n)

    def compute_similar_music_for_artist(self, artist_id, similarity_func,n):
        """
        artist_id : the artist id
        similarity_func =the similarity function 
        n : number of similar values to return 

        this method takes the artist name, the similarity_metrics and number of similar 
        music to return. This values are then feed to the compute similar method and return result gotten from 
        the compute_similar method.
        
        """
        return self._compute_similar(types="music for artist", iden=artist_id, similarity_func=similarity_func, n=n)



In [9]:

def main(query1, query2, query3, query4):
    """
    query1 : This represent the kind of service you want
    query2 : This represent the id of the music or artist 
    query3 : This represent the kind of similarity metrics you want to use
    query4 : This represent the number of values to return 

    This is the main function and it takes different values and return the result as a dataframe 
    """

    query4 = int(query4)
    query2 = int(query2)
    simil = Similarity_comp()
    similar_dict = {"1": "euclidean", "2" :"cosine", "3" :"correlation", "4" :"hamming","5" : "manhattan"}
    if query1 == "1":
        simil.compute_similar_music(music_id=query2, similarity_func=similar_dict[query3], n=query4)
    if query1 == "2":
        simil.compute_similar_artist(artist_id=query2, similarity_func=similar_dict[query3], n=query4)
    if query1 == "3":
        simil.compute_similar_music_for_artist(artist_id=query2, similarity_func=similar_dict[query3], n=query4)

        
    

### The main function the implementaion takes place 

In [1]:
       
if __name__ == "__main__":
    print("Thank you for using our recommendation engine")
    query1 = input("We offer three different services please choose?\n\
                    Enter 1 for finding top similar music\n\
                    Enter 2 for finding top similar artist\n\
                    Enter 3 for finding top similar music based on a particular artist\n")
    if query1 == "1":
        query2 = input("Enter the music id: ")
    elif query1 == "2" or query1 == "3":
        query2 = input("Enter the artist id: ")
    query3 = input("What similarity metric should we use?\n\
                    Enter 1 for euclidean\n\
                    Enter 2 for cosine\n\
                    Enter 3 for correlation\n\
                    Enter 4 for jaccard coefficient\n\
                    Enter 5 for manhattan\n")
    if query1 == "1":
        query4 = input("How many similar music should we return  ")
    elif query1 == "2":
        query4 = input("How many similar artist should we return  ")
    elif query1 == "3":
        query4 = input(f"How many similar music should we return for {query2}  ")
    main(query1, query2, query3, query4)
    

Thank you for using our recommendation engine
We offer three different services please choose?
                    Enter 1 for finding top similar music
                    Enter 2 for finding top similar artist
                    Enter 3 for finding top similar music based on a particular artist
1
Enter the music id: 6KuQTIu1KoTTkLXKrwlLPV
What similarity metric should we use?
                    Enter 1 for euclidean
                    Enter 2 for cosine
                    Enter 3 for correlation
                    Enter 4 for jaccard coefficient
                    Enter 5 for manhattan
2
How many similar music should we return  5


NameError: name 'main' is not defined